Напишите скрипт, принимающий на вход Excel-спредшит с данными как в приложенном файле test_input.xlsx и генерирующий из этих данных XML-файл со структурой, как в приложенном файле output_xml.xml. Для простоты просмотра файл output_xml.xml содержит только две строчки данных из excel-файла, Ваш результирующий файл должен содержать данные из всех строчек исходного Excel-файла. Обратите внимание в каком формате написаны даты, названия и денежные выражения в результирующем файле. Форматы значений в вашем файле должны совпадать.

In [14]:
import pandas as pd
import xml.etree.ElementTree as ET

df = pd.read_excel('test_input.xlsx', skiprows=4)

certdata = ET.Element('CERTDATA')
filename = ET.SubElement(certdata, 'FILENAME')
filename.text = 'SABR000000119042023002'
envelope = ET.SubElement(certdata, 'ENVELOPE')

for index, row in df.iterrows():
    ecert = ET.SubElement(certdata, 'ECERT')

    certno = ET.SubElement(ecert, 'CERTNO')
    certno.text = str(row['Ref no'])

    certdate = ET.SubElement(ecert, 'CERTDATE')
    certdate.text = row['Issuance Date'].strftime('%Y-%m-%d')

    status = ET.SubElement(ecert, 'STATUS')
    status.text = str(row['Status'])

    iec = ET.SubElement(ecert, 'IEC')
    iec.text = str(row['IE Code'])

    expname = ET.SubElement(ecert, 'EXPNAME')
    expname.text = '"' + str(row['Client']) + '"'

    billid = ET.SubElement(ecert, 'BILLID')
    billid.text = str(row['Bill Ref no'])

    sdate = ET.SubElement(ecert, 'SDATE')
    sdate.text = row['SB Date'].strftime('%Y-%m-%d')

    scc = ET.SubElement(ecert, 'SCC')
    scc.text = str(row['SB Currency'])

    svalue = ET.SubElement(ecert, 'SVALUE')
    svalue.text = str(row['SB Amount'])

tree = ET.ElementTree(certdata)
tree.write('output_mekhtieva_1.xml', encoding='utf-8', xml_declaration=True)

Модифицируйте скрипт из задания 1 так, чтобы он заполнял для каждой строки данных
(т.е. для каждого элемента ECERT результирующего файла) дополнительный атрибут
SVALUEUSD. Скрипт должен открывать сайт ЦБ РФ
(https://www.cbr.ru/currency_base/daily/) и вытягивать официальный курс доллара США за
дату, указанную в колонке “SB Date” эксель-файла (соответствует атрибуту SDATE XML-
файла), делить значение атрибута SVALUE на курс доллара США, округлять результат до
сотых долей и записывать результат в атрибут SVALUEUSD. Для простоты можно
считать, что в атрибуте SVALUE всегда будет значение в рублях.
Результирующий файл должен иметь такую же структуру как вложенный файл
output_xml_v2.xml

In [16]:
import requests
from bs4 import BeautifulSoup

def get_usd_exchange_rate(date):
    url = f'https://www.cbr.ru/currency_base/daily/?UniDbQuery.Posted=True&UniDbQuery.To={date.strftime("%d.%m.%Y")}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    rows = soup.find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        if len(columns) == 5 and columns[1].text == 'USD':
            return float(columns[4].text.replace(',', '.'))
    return None

for index, row in df.iterrows():
    ecert = ET.SubElement(envelope, 'ECERT')

    certno = ET.SubElement(ecert, 'CERTNO')
    certno.text = str(row['Ref no'])

    certdate = ET.SubElement(ecert, 'CERTDATE')
    certdate.text = row['Issuance Date'].strftime('%Y-%m-%d')

    status = ET.SubElement(ecert, 'STATUS')
    status.text = str(row['Status'])

    iec = ET.SubElement(ecert, 'IEC')
    iec.text = str(row['IE Code'])

    expname = ET.SubElement(ecert, 'EXPNAME')
    expname.text = '"' + str(row['Client']) + '"'

    billid = ET.SubElement(ecert, 'BILLID')
    billid.text = str(row['Bill Ref no'])

    sdate = ET.SubElement(ecert, 'SDATE')
    sdate.text = row['SB Date'].strftime('%Y-%m-%d')

    scc = ET.SubElement(ecert, 'SCC')
    scc.text = str(row['SB Currency'])

    svalue = ET.SubElement(ecert, 'SVALUE')
    svalue.text = str(row['SB Amount'])

    usd_rate = get_usd_exchange_rate(row['SB Date'])
    if usd_rate is not None:
        svalue_usd = round(float(row['SB Amount']) / usd_rate, 2)
        svalue_usd_elem = ET.SubElement(ecert, 'SVALUEUSD')
        svalue_usd_elem.text = str(svalue_usd)

tree = ET.ElementTree(certdata)
tree.write('output_mekhtieva_2.xml', encoding='utf-8', xml_declaration=True)